In [97]:
import os
root_dir_name = 'AmazonProductSearch'
curr_dir = os.getcwd()
root_dir = curr_dir[:curr_dir.index(root_dir_name) + len(root_dir_name)]
dataset_dir = os.path.join(root_dir , 'Dataset')
en_dataset_dir = os.path.join(dataset_dir ,'English')


In [98]:
import pandas as pd 
df_main = pd.read_parquet(os.path.join(en_dataset_dir , 'finalized_dataset_forvectorization.parquet'))
df_queries = pd.read_parquet(os.path.join(dataset_dir , 'shopping_queries_dataset_examples.parquet'))
df_queries = df_queries[df_queries['product_locale'] == 'us']
df_queries

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2618565,2618565,자전거트레일러,130539,B06XTZYJL3,us,I,1,1,train
2618566,2618566,자전거트레일러,130539,B010LLGWL8,us,E,1,1,train
2618567,2618567,자전거트레일러,130539,B010LLGWKE,us,E,1,1,train
2618568,2618568,자전거트레일러,130539,B003CUBPUY,us,I,1,1,train


In [113]:
df_queries_h = df_queries[df_queries['small_version'] == 1]
df_queries_e = df_queries[df_queries['small_version'] == 0]

In [114]:
len(df_queries_e['query_id'].unique()), len(df_queries_h['query_id'].unique())

(67508, 29844)

In [115]:
trained_product_ids = df_main['product_id']

planning to consider a final of 10k easy , 5k hard - so give it around 33% of selection process

In [116]:
(df_queries_e[(df_queries_e['product_id'].isin(trained_product_ids)) & (df_queries_e['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 5).sum() , \
 (df_queries_h[(df_queries_h['product_id'].isin(trained_product_ids)) & (df_queries_h['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 3).sum()

(31569, 15728)

Stargies query selection process as we have removed a lot of products in the data preprocessing step

In [117]:
easy_sampled_query = (df_queries_e[(df_queries_e['product_id'].isin(trained_product_ids)) & (df_queries_e['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 5)
easy_sampled_query_index = easy_sampled_query.index[easy_sampled_query.values]
hard_sampled_query = (df_queries_h[(df_queries_h['product_id'].isin(trained_product_ids)) & (df_queries_h['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 3)
hard_sampled_query_index = hard_sampled_query.index[hard_sampled_query.values]

In [118]:
len(easy_sampled_query_index) , len(hard_sampled_query_index)

(31569, 15728)

In [119]:
df_queries_e = df_queries_e[df_queries_e['query_id'].isin(easy_sampled_query_index)]
df_queries_h = df_queries_h[df_queries_h['query_id'].isin(hard_sampled_query_index)]

In [120]:
# randomly selection of 10 K queries from each easy and hard a with random_state = 42
import random
random.seed(42)
easy_search_queries = random.sample(list(df_queries_e['query_id'].unique()) , 10000)
hard_search_queries = random.sample(list(df_queries_h['query_id'].unique()) , 5000)
easy_search_queries.sort()
hard_search_queries.sort()

In [121]:
len(easy_search_queries) , len(hard_search_queries)

(10000, 5000)

In [122]:
df_queries_e = df_queries_e[df_queries_e['query_id'].isin(easy_search_queries)]
df_queries_h = df_queries_h[df_queries_h['query_id'].isin(hard_search_queries)]

In [123]:
df_queries_e

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
1370,1370,#8 phillips head wood screws,43,B07D1L98GC,us,S,0,1,train
1371,1371,#8 phillips head wood screws,43,B07PQS4G74,us,E,0,1,train
1372,1372,#8 phillips head wood screws,43,B07PNHNL89,us,E,0,1,train
1373,1373,#8 phillips head wood screws,43,B07L4BSVDV,us,E,0,1,train
1374,1374,#8 phillips head wood screws,43,B07GDQ9LRN,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2259800,2259800,zucchini noodle maker spiralizer,115890,B00WKENSDA,us,E,0,1,test
2259801,2259801,zucchini noodle maker spiralizer,115890,B00L0R9R38,us,E,0,1,test
2259802,2259802,zucchini noodle maker spiralizer,115890,B00J19AR7W,us,E,0,1,test
2259803,2259803,zucchini noodle maker spiralizer,115890,B00IIVRB3W,us,E,0,1,test


In [124]:
len(df_queries_e['query_id'].unique()) , len(df_queries_h['query_id'].unique())

(10000, 5000)

In [125]:
df_queries_h

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
149,149,# 10 self-seal envelopes without window,5,B07RP3F2LW,us,E,1,1,train
150,150,# 10 self-seal envelopes without window,5,B07FK9PCZB,us,E,1,1,train
151,151,# 10 self-seal envelopes without window,5,B07FMDV5LR,us,S,1,1,train
152,152,# 10 self-seal envelopes without window,5,B07GJCSF8Z,us,E,1,1,train
153,153,# 10 self-seal envelopes without window,5,B07GJKFC5X,us,E,1,1,train
...,...,...,...,...,...,...,...,...,...
2259720,2259720,zte axon m case,115885,B075VDP82B,us,E,1,1,train
2259721,2259721,zte axon m case,115885,B071YCYN42,us,E,1,1,train
2259722,2259722,zte axon m case,115885,B01ER35FA0,us,E,1,1,train
2259723,2259723,zte axon m case,115885,B00MNMT78Y,us,E,1,1,train


In [126]:
df_queries_e.to_parquet(os.path.join(en_dataset_dir , 'easy_search_query_dataset.parquet'))
df_queries_h.to_parquet(os.path.join(en_dataset_dir , 'hard_search_query_dataset.parquet'))


In [129]:
len(df_queries_e['query_id'].unique()) , len(df_queries_h['query_id'].unique())

(10000, 5000)